# RecStudio4Industry 训练教程

RecStudio4Industry 是一个面向工业互联网的推荐系统开发工具包，它继承了RecStudio的模块化设计理念，旨在通过对推荐系统模型进行模块化设计，帮助算法工程师快速的进行各类模块的尝试。另外，它在RecStudio之外，额外添加了工业场景友好的特色功能，支持快速搭建工业互联网推荐系统并部署，工业友好的主要特色功能如下：

- 支持HDFS系统训练数据读取。相比于学术界使用的小批量数据集，工业级的数据往往是十分庞大的，需要按天存取到HDFS分布式文件系统中。因此，RecStudio4Industry 提供了HDFS数据读取接口方便快速对接工业场景数据。
- 模型分布式训练。工业庞大的数据量往往对训练时间提出了更高的要求，因此RecStudio4Industry 提供了分布式训练接口方便工业级推荐系统模型快速进行分布式训练。
- 支持工业级推荐系统部署。RecStudio4Industry 提供了工业级推荐系统部署接口，支持将训练好的模型快速部署到工业互联网的多阶段推荐系统中。其中包括基于Redis的数据缓存等。

下面教程将详细介绍如何使用RecStudio4Industry进行模型训练，包括以下几个方面的详细需求：

1. 训练数据的配置
2. 模型配置以及自定义模型搭建
3. 模型的单机训练和分布式多机训练
4. 模型的保存和读取

## 训练数据的配置


由于工业推荐系统数据量巨大，往往采用分布式的系统进行存取，常用的分布式系统为HFDS。RecStudio4Industry支持将数据存入HFDS，并使用HFDS进行训练。下面我们以KuaiShou发布的RecFlow全链路推荐数据集为例，介绍数据的组织形式。

1. 用户数据日志。这部分数据一般用于记录用户对物品的交互行为，如点击、收藏等。具体地，用户在刷视频时每一次刷新对应地向系统发送一次请求，系统会对该请求进行算法处理。系统是多阶段漏斗形状的，最终返回给用户10-20个候选物品，而后用户对这些物品的交互行为会被回传到系统，形成一条数据。往往包括：请求ID，用户ID，用户特征，物品ID，物品特征，用户历史行为，交互时间等。由于用户数据日志很多，大多数情况下会按照以天为单位进行文件划分，如2019-07-01.csv、2019-07-02.csv等。在RecFlow数据集中，每天的用户数据日志被存为xxx.feather文件。
2. 物品数据文件。这部分数据往往包括了平台上所有物品的信息，组织成KV表的形式，其中key为物品ID，value为该物品的一系列特征：如短视频平台上：视频的创作者、视频的分类、视频时长、视频标签等。
3. 用户行为序列文件。这部分数据存储了用户的行为序列，表示用户的历史交互，组织成KV表的形式，其中key为请求ID，value为该请求ID对应的用户的行为序列。


数据集的配置文件模板如下：

```json
{
    "name": "数据集名称 (必选)",
    "type": "数据集类型，如 hdfs 或者 file (必选)",
    "url": "数据集交互数据的位置，如hdfs://127.0.0.1:8020/recstudio/recflow/daily_logs (必选)",
    "file_format": "数据存储格式，如 feather, parquet, pkl等。默认为 auto, 即自动检测。",
    "date_format": "时间格式，如 %Y-%m-%d。默认为 %Y-%m-%d。",
    "item_col": "物品id列名 (必选)",
    "context_features": ["使用的上下文特征列表", "特征1", "特征2", "(必选)"],
    "item_features": ["使用的物品特征列表", "context_features 和 item_features 不能重复, 均为主表的列名", "(必选)"],
    "labels": ["使用的标签列表", "多个标签一般表示多任务训练", "标签列表不能为空", "(必选)"],
    "filter_settings": {
        "过滤特征名称": ["过滤条件1", "过滤条件2", "过滤条件形式为 (==, !=, >=, <=, >, <)[number]"],
        "like": ["==1"],
        "用途": "一般用于按照标签过滤，比如召回模型需要只需要保留label=1的样本，负样本来自于从候选物品集采样"
    },
    "item_info": {
        "url": "召回模型候选物品信息数据存储位置，如 hdfs://127.0.0.1:8020/recstudio/recflow/others/video_info.pkl, 召回模型必选",
        "key": "item id 的列名。对于dataframe形式的文件必须提供, 对于dict形式的文件不需要提供",
        "columns": ["物品特征表的列名", "在item_info中必选", "尤其用于针对dict文件的特征命名"],
        "use_cols": ["item_info中需要使用的特征列表", "如果为空，则使用所有列"]
    },
    "user_sequential_info": {
        "url": "用户序列数据存储位置，如 hdfs://127.0.0.1:8020/recstudio/recflow/daily_logs。设置 user_sequential_info 为 null 则表示不使用独立的序列文件",
        "file_format": "序列数据存储格式，如 feather, parquet, pkl 等。默认为 auto, 即自动检测。",
        "key": "查询序列数据索引的键值，如 request_id。该值还需要在交互数据表中存在。",
        "columns": ["序列特征表的列名", "在item_info中必选", "尤其用于针对dict文件的特征命名", "一般与item_features相同或者为其子集"],
        "use_cols": ["user_sequential_info 中需要使用的特征列表", "如果为空，则使用所有列"]
    },
    "stats": {
        "特征1": 6,
        "特征2": 10,
        "(必选)": "离散特征的数量"
    },
    "train_settings": {
        "start_date": "2024-01-13 (必选)，训练数据的开始日期",
        "end_date": "2024-02-08 , 训练数据的结束日期。不包含该日期的数据(必选)"
    },
    "test_settings": {
        "start_date": "2024-02-08 (必选)，测试数据的开始日期",
        "end_date": "2024-02-09，测试数据的结束日期。不包含该日期的数据 (必选)"
    }
}
```

具体的，RecFlow数据集在用于召回模型的训练时，数据配置如下：（值得注意的是，召回模型往往只在曝光数据上训练，因此需要设置filtering_settings）
```json
{
    "name": "recflow",
    "type": "hdfs",
    "url": "hdfs://node1:8020/recstudio/recflow/realshow",
    "item_col": "video_id",
    "context_features": ["user_id", "device_id", "age", "gender", "province"],
    "item_features": ["video_id", "author_id", " category_level_two", "upload_type", "category_level_one"],
    "labels": ["like"],
    "filter_settings": {
        "effective_view": ["==1"]
    },
    "item_info": {
        "url": "hdfs://node1:8020/recstudio/recflow/others/video_info.pkl",
        "key": "video_id",
        "columns": ["video_id", "author_id", "category_level_two", "upload_type", "upload_timestamp", "category_level_one"],
        "use_cols": ["video_id", "author_id", "category_level_two", "upload_type", "category_level_one"]
    },
    "user_sequential_info": {
        "url": "hdfs://node1:8020/recstudio/recflow/seq_effective_50",
        "key": "request_id",
        "columns": ["video_id", "author_id", "category_level_two", "category_level_one", "upload_type", "upload_timestamp", "duration", "request_timestamp", "playing_time", "request_id"],
        "use_cols": ["video_id", "author_id", "category_level_two", "category_level_one", "upload_type"]
    },
    "stats": {
        "request_id": 9370581,
        "user_id": 42472,
        "device_id": 42561,
        "age": 8,
        "gender": 3,
        "province": 79,
        "video_id": 82216301,
        "author_id": 33474011,
        "category_level_one": 140,
        " category_level_two": 784,
        "upload_type": 40
    },
    "train_settings": {
        "start_date": "2024-01-13",
        "end_date": "2024-02-18"
    },
    "test_settings": {
        "start_date": "2024-02-18",
        "end_date": "2024-02-19"
    }
}
```

至此，数据集配置文件即为完成。后续RecStudio4Industry可根据配置自动生成DataLoader。

## 模型配置以及自定义模型的搭建
本章节将描述如何使用库内已实现模型进行训练以及如何继承基类进行自定义模型的搭建。因此将分为两个子章节进行介绍。

### 使用内置模型进行训练


1. 首先，需要clone RecStudio4Industry到本地，并安装依赖。

    ```bash
    pip install -r requirements.txt
    ```
2. 添加rs4industry到python的路径，以方便调用。

    ```bash
    export PYTHONPATH=$PYTHONPATH:/path/to/recstudio-industry
    ```

3. 配置模型的配置文件，其中用于定义模型的结构参数，如embedding size， hidden size等。示例如下：

    ```json
    {
        "embedding_dim": 8,
        "mlp_layers": [128, 128],
        "prediction_layers": [32],
        "activation": "relu",
        "dropout": 0.3,
        "batch_norm": false
    }
    ```

4. 配置训练的参数，其中用于定义训练的超参数，如batch size， learning rate等。示例如下：

    ```json
    {
        "epochs": 1,
        "train_batch_size": 2048,
        "eval_batch_size": 4096,
        "optimizer": "adam",
        "learning_rate": 1e-2,
        "checkpoint_dir": "saves/ranker",
        "checkpoint_steps": 1000,
        "evaluation_strategy": "epoch",
        "eval_interval": 1,
        "metrics": ["auc", "logloss"],
        "earlystop_metric": "auc"
    }
    ```
    
更多的配置参数说明可以参考 [配置参数](../../../rs4industry/config/training.py)。

5. 新建python代码脚本，利用rs4industry导入数据集、模型并进行训练。

In [ ]:
# train.py
from rs4industry.data.dataset import get_datasets
from rs4industry.model.rankers import MLPRanker
from rs4industry.trainer import Trainer

data_config_path = "../../../examples/config/data/recflow_ranker.json"
train_config_path = "../../../examples/config/mlp_ranker/train.json"
model_config_path = "../../../examples/config/mlp_ranker/model.json"

(train_data, eval_data), data_config = get_datasets(data_config_path)

model = MLPRanker(data_config, model_config_path)

trainer = Trainer(model, train_config_path)

trainer.fit(train_data, eval_data)


6. 至此，模型的训练脚本已经完成，可以运行该脚本进行模型的训练。使用python指令直接执行该指令则会默认使用单机单卡训练。如果需要单机多卡或者多机多卡训练，可以参考[分布式训练](./quickstart_ZH.ipynb#模型的单机训练和分布式多机训练)进行配置。

    ```bash
    python train.py
    ```

### 自定义模型进行训练

本章节将展示如何通过继承基类，自定义模型进行训练。我们将分别展示召回模型和精排模型的自定义用法，以展示双塔模型和单塔模型需要配置的接口。

首先，前面两步和使用内置模型的做法一样，需要配置环境：
1. 首先，需要clone RecStudio4Industry到本地，并安装依赖。


In [ ]:
!pip install -r requirements.txt  

2. 添加rs4industry到python的路径，以方便调用。

In [ ]:
!export PYTHONPATH=$PYTHONPATH:/path/to/recstudio-industry


#### 召回模型（粗排模型）

3. 导入BaseRetriever类，并继承BaseRetriever类，实现自定义模型。召回模型主要分四个模块组成：

    - query_encoder：上下文（查询）特征编码器，用于将用户侧和上下文侧特征编码为向量表示。
    - item_encoder: 物品特征编码器，用于将物品侧特征编码为向量表示。
    - score_function: 评分函数，用于计算用户-物品对之间的匹配程度。
    - loss_function: 损失函数，用于计算模型预测值和真实标签之间的差异。

因此，需要override以下方法。其中定义模型结构时候需要的配置参数都是来自于model.json文件。

In [ ]:
import torch
from collections import OrderedDict
from rs4industry.model.retriever import BaseRetriever
from rs4industry.model.utils import get_modules
from rs4industry.model.module import MultiFeatEmbedding, MLPModule

class MYMLPRetriever(BaseRetriever):
    def __init__(self, config, *args, **kwargs):
        super().__init__(config, *args, **kwargs)

    def get_item_encoder(self):
        item_emb = MultiFeatEmbedding(
            features=self.data_config.item_features,
            stats=self.data_config.stats,
            embedding_dim=self.model_config.embedding_dim,
            concat_embeddings=True
        )
        mlp = MLPModule(
            mlp_layers= [item_emb.total_embedding_dim] + self.model_config.mlp_layers,
            activation_func=self.model_config.activation,
            dropout=self.model_config.dropout,
            bias=True,
            batch_norm=self.model_config.batch_norm,
            last_activation=False,
            last_bn=False
        )
        return torch.nn.Sequential(OrderedDict([
            ("item_embedding", item_emb),
            ("mlp", mlp)
            ]))
    

    def get_query_encoder(self):
        context_emb = MultiFeatEmbedding(
            features=self.data_config.context_features,
            stats=self.data_config.stats,
            embedding_dim=self.model_config.embedding_dim
        )
        base_encoder = get_modules("encoder", "BaseQueryEncoderWithSeq")(
            context_embedding=context_emb,
            item_encoder=self.item_encoder
        )
        output_dim = self.model_config.mlp_layers[-1] + context_emb.total_embedding_dim
        mlp = MLPModule(
            mlp_layers= [output_dim] + self.model_config.mlp_layers,
            activation_func=self.model_config.activation,
            dropout=self.model_config.dropout,
            bias=True,
            batch_norm=self.model_config.batch_norm,
            last_activation=False,
            last_bn=False
        )

        return torch.nn.Sequential(OrderedDict([
            ("encoder", base_encoder),
            ("mlp", mlp)
            ]))

    def get_score_function(self):
        return get_modules("score", "InnerProductScorer")()
    
    def get_loss_function(self):
        return get_modules("loss", "BPRLoss")()
    
    def get_negative_sampler(self):
        sampler_cls = get_modules("sampler", "UniformSampler")
        return sampler_cls(num_items=self.data_config.num_items)

4. 而后，和训练内置模型一致，通过根据数据集、模型和训练配置文件，即可利用rs4industry快速完成训练脚本：

In [ ]:
# train.py
from rs4industry.data.dataset import get_datasets
from rs4industry.trainer import Trainer

data_config_path = "../../../examples/config/data/recflow_retriever.json"
train_config_path = "../../../examples/config/mlp_retriever/train.json"
model_config_path = "../../../examples/config/mlp_retriever/model.json"

(train_data, eval_data), data_config = get_datasets(data_config_path)

model = MYMLPRetriever(data_config, model_config_path)

trainer = Trainer(model, train_config_path)

trainer.fit(train_data, eval_data)

#### 精排模型

和召回模型不同的地方是，精排模型往往结构是单塔结构，通常关注特征之间的交叉，特征之间的组合。因此需要override的函数也不同，需要构建的模块有：

- 序列特征聚合器：用于将形状为(L,D)的特征序列聚合为形状为(D)的单个特征，以用于后续特征交互
- 特征交叉模块：用于将一系列特征进行交互，通常为序列特征聚合器输出的单个特征。常用的模块如MLP、FM等
- 预测模块：用于对交叉后的特征进行最后的预测，通常为全连接层，接在特征交叉模块后面
- 损失函数：用于计算预测值和真实标签之间的loss

3. 导入BaseRanker类，并继承BaseRanker类，实现自定义模型：

In [ ]:
import torch
from rs4industry.model.ranker import BaseRanker
from rs4industry.model.utils import get_modules
from rs4industry.model.module import MLPModule, LambdaModule


class MYMLPRanker(BaseRanker):
    def get_sequence_encoder(self):
        cls = get_modules("module", "AverageAggregator")
        encoder = cls(dim=1)
        return encoder
    
    def get_feature_interaction_layer(self):
        flatten_layer = LambdaModule(lambda x: x.flatten(start_dim=1))  # [B, N, D] -> [B, N*D]
        mlp_layer = MLPModule(
            mlp_layers= [self.num_feat * self.model_config.embedding_dim] + self.model_config.mlp_layers,
            activation_func=self.model_config.activation,
            dropout=self.model_config.dropout,
            bias=True,
            batch_norm=self.model_config.batch_norm,
            last_activation=False,
            last_bn=False
        )
        return torch.nn.Sequential(flatten_layer, mlp_layer)
    
    def get_prediction_layer(self):
        pred_mlp = MLPModule(
            mlp_layers=[self.model_config.mlp_layers[-1]] + self.model_config.prediction_layers + [1],
            activation_func=self.model_config.activation,
            dropout=self.model_config.dropout,
            bias=True,
            batch_norm=self.model_config.batch_norm,
            last_activation=False,
            last_bn=False
        )
        return pred_mlp

    def get_loss_function(self):
        return get_modules("loss", "BCEWithLogitLoss")(reduction='mean')

4. 而后，和训练内置模型一致，通过根据数据集、模型和训练配置文件，即可利用rs4industry快速完成训练脚本

In [ ]:
# train.py
from rs4industry.data.dataset import get_datasets
from rs4industry.trainer import Trainer

data_config_path = "../../../examples/config/data/recflow_ranker.json"
train_config_path = "../../../examples/config/mlp_ranker/train.json"
model_config_path = "../../../examples/config/mlp_ranker/model.json"

(train_data, eval_data), data_config = get_datasets(data_config_path)

model = MYMLPRanker(data_config, model_config_path)

trainer = Trainer(model, train_config_path)

trainer.fit(train_data, eval_data)

## 模型的单机训练和分布式多机训练

RecStudio4Industry支持基础的单机单卡训练、单机多卡训练和分布式训练，下面分别介绍如何使用RecStudio4Industry进行单机单卡训练、单机多卡训练和分布式训练。

1. 单机单卡训练：直接使用python指令启动或者可以使用accelerate命令启动(accelerate示例配置文件参考[single_gpu.json](../../../examples/distributed_training/single_gpu.json)

    ```shell
    #  使用python指令启动
    CUDA_VISIBLE_DEVICES=1 python main.py
    # 使用accelerate命令启动
    accelerate launch --config_file single_gpu.json main.py
    ```

2. 单机多卡训练：先进行单机多卡的配置，示例文件参考[配置文件single_node.json](../../../examples/distributed_training/single_node.json)。而后使用accelerate命令启动。

    ```shell
    accelerate launch --config_file single_node.json main.py
    ```

    注意，单机多卡训练会默认占用本机端口29500。如果需要起多个任务，需要在命令中或者json文件中指定不同的端口号：`--main_process_port 29501` （命令行指定） 或者 "main_process_port": 29501 （json 文件）。

    另外，单机多卡以及多机多卡训练目前采用的都是DistributedDataParallel（DDP）的训练方式，在训练过程中，每个进程都会在对应的GPU上保存一份完整的模型、优化器，另外还会在每个GPU上维护“bucket”用于收集训练时候来自其他卡的gradient，因此在模型prepare时候会占用两倍模型大小的显存开销。详情请参考[blog1](https://discuss.pytorch.org/t/memory-consumption-for-the-model-get-doubled-after-wrapped-with-ddp/130837), [blog2](https://medium.com/deep-learning-for-protein-design/a-comprehensive-guide-to-memory-usage-in-pytorch-b9b7c78031d3).

3.  多机多卡分布式训练：
    - 多台机器上都进行环境配置，下载recstudoudio4industry，并安装依赖。
    - 每台机器上进行多机多卡的配置，示例文件参考 [配置文件 multi_node_rank0.json](../../../examples/distributed_training/multi_nodes_rank0.json) 和 [配置文件 multi_node_rank1.json](../../../examples/distributed_training/multi_nodes_rank1.json)。而后先使用accelerate命令在rank0机器上启动，再依次启动其他机器：
    
    ```shell
    accelerate launch --config_file multi_node_rank0.json main.py
    ```


注意，以上所有accelerate的配置文件都可以自己通过accelerate命令生成，即：

  ```shell
  accelerate config --config_file xxx.json
  ```

而后根据需求在交互式的命令行中选择对应的配置。更多详细的解释用法参考[accelerate](https://github.com/huggingface/accelerate)文档。

## 模型的存储和读取

模型训练的checkpoint将被自动存到训练配置文件的`checkpoint_dir`目录下。包括模型各个阶段的权重文件和配置，最优模型的权重文件和配置以及日志文件。这里我们展示如何根据路径加载模型并进行离线评估。

In [ ]:
from rs4industry.model.base import BaseModel
from rs4industry.data.dataset import get_datasets
from rs4industry.trainer import Trainer

data_config_path = "../../../examples/config/data/recflow_ranker.json"
train_config_path = "../../../examples/config/mlp_ranker/train.json"

(train_data, eval_data), data_config = get_datasets(data_config_path)
model = BaseModel.from_pretrained("saves/mlp_ranker/best_ckpt")
print(model)

trainer = Trainer(model, train=False)

trainer.config.metrics = ["auc", "logloss"]
trainer.config.eval_batch_size = 2048

trainer.evaluation(eval_data)
